# 블록체인프로그래밍 4주차 과제 201810825 홍종현

# 연습문제 1

In [8]:
import hashlib
txA='Hello'
txB='How are you?'

### 1-1 txA을 재해싱하세요(\_hA)

In [13]:
_hA=hashlib.sha256(hashlib.sha256(txA.encode()).digest()).hexdigest()
print("재해싱된 결과 :",_hA)

재해싱된 결과 :  70bc18bef5ae66b72d1995f8db90a583a60d77b4066e4653f1cead613025861c


### 1-2 \_hA를 바이트교환을 하세요 hA

In [22]:
hA="".join(reversed([_hA[i:i+2] for i in range(0, hashlib.sha256(txA.encode()).digest_size*2, 2)]))
print("바이트교환된 결과 :",hA)

바이트교환된 결과 : 1c86253061adcef153466e06b4770da683a590dbf895192db766aef5be18bc70


### 1-3 txB를 재해싱하세요 (\_hB)

In [16]:
_hB=hashlib.sha256(hashlib.sha256(txB.encode()).digest()).hexdigest()
print("재해싱된 결과 :",_hB)

재해싱된 결과 : 1027ce357faf49fc632d35908db181ae951aa3df9060595d94fb33769d71e7d0


### 1-4 \_hB를 바이트교환을 하세요 hB

In [24]:
hB="".join(reversed([_hB[i:i+2] for i in range(0, hashlib.sha256(txA.encode()).digest_size*2, 2)]))
print("바이트교환된 결과 :",hB)

바이트교환된 결과 : d0e7719d7633fb945d596090dfa31a95ae81b18d90352d63fc49af7f35ce2710


### 1-5 hA + hB를 하세요 \_hAhB

In [27]:
_hAhB=hA+hB
print("_hAhB :",_hAhB)

_hAhB : 1c86253061adcef153466e06b4770da683a590dbf895192db766aef5be18bc70d0e7719d7633fb945d596090dfa31a95ae81b18d90352d63fc49af7f35ce2710


### 1-6 \_hAhB를 재해싱하세요 (\_hAB)

In [30]:
_hAB=hashlib.sha256(hashlib.sha256(_hAhB.encode()).digest()).hexdigest()
print("재해싱된 결과 :",_hAB)

재해싱된 결과 : e0c76d87a5a5c18ab29757603c5d1bda709306203b0a44c53fc6c90fba162903


### 1-7 \_hAB를 바이트교환을 하세요 hAB

In [33]:
hAB="".join(reversed([_hAB[i:i+2] for i in range(0, 32*2, 2)]))
print("바이트교환된 결과 :",hAB)

바이트교환된 결과 : 032916ba0fc9c63fc5440a3b20069370da1b5d3c605797b28ac1a5a5876dc7e0


# 연습문제 2
#### 친구에게  주소를 구하고, 그 주소로 송금해 보자. 송금이 되지 않으면 왜 안되는지 이유를 알아보자.
#### 안되면 친구의 주소를 만들고 전송한다.
#### 잔고의 증가분을 출력하세요.
#### 소요된 gas비용 출력하세요.

In [111]:
#친구의 enode : "enode://dd2979bb56bcbc3a3d5955a53e482d1236bd49bdb9834f5b73cfa7b8584f77630ea5d1026b6cc4f5417ede6f5e5b62ffe8c3077ac8251671056970544c0313d3@175.193.72.238:38445"
#친구의 주소 : 0x40944b4662569d4099e55e77266075f017b8701d
!geth --exec "admin.addPeer('enode://dd2979bb56bcbc3a3d5955a53e482d1236bd49bdb9834f5b73cfa7b8584f77630ea5d1026b6cc4f5417ede6f5e5b62ffe8c3077ac8251671056970544c0313d3@175.193.72.238:38445')" attach http://localhost:8445 
!geth --exec "eth.getBalance('0x40944b4662569d4099e55e77266075f017b8701d');" attach http://localhost:8445 

true
0


#### 친구의 enode를 알아내어 peer에 추가 시킨뒤 잔고를 확인하였으나 0이 아님에도 0으로 출력됨
#### 추가적인 공부를 통하여 몇가지 원인을 탐색
#### 1. peer는 정적IP를 가지고 있어야하지만, 나와 내 친구의 IP는 동적IP
#### 2. 네트워크 문제가 있어 연결이 안될 수 있음 (특히 방화벽이 막는 경우)
#### 3. OS의 시간이 맞지 않는 경우

In [3]:
%%writefile src/transact.js
miner.setEtherbase(eth.accounts[0]);
var bal1=eth.getBalance(eth.coinbase);
var bal2=eth.getBalance(eth.accounts[1]);
console.log('sender balance in ether: ', web3.fromWei(bal1,"ether"));
console.log('receiver balance in ether: ', web3.fromWei(bal2,"ether"));
var h=eth.sendTransaction({from:eth.coinbase, to:eth.accounts[1],value:10000});
miner.start(1);admin.sleepBlocks(1);miner.stop();
var bal1new=eth.getBalance(eth.coinbase);
var bal2new=eth.getBalance(eth.accounts[1]);
console.log('- New sender balance in ether: ', web3.fromWei(bal1new,"ether"));
console.log('- New receiver balance in ether: ', web3.fromWei(bal2new,"ether"));
console.log('- Used Gas:', eth.getTransactionReceipt(h).gasUsed);

Overwriting src/transact.js


In [4]:
!geth --exec "loadScript('src/transact.js')" attach http://localhost:8445 

sender balance in ether:  1294.99999999999972
receiver balance in ether:  1800.00000000000027
- New sender balance in ether:  1304.99999999999971
- New receiver balance in ether:  1800.00000000000028
- Used Gas: 21000
true


# 연습문제 3
블록헤더 데이터의 해시 값에 NONCE를 증가시키면서 앞 자리의 0의 개수를 맞출 때까지 반복한다.

In [131]:
!geth --exec "eth.protocolVersion" attach http://localhost:8445

"0x3f"


In [136]:
!geth --exec "eth.blockNumber" attach http://localhost:8445

1204


### 3-1 찾고자 하는 해시가 ```0000```로 시작한다고 하자. 몇 회만에 찾는지 출력하세요.

In [138]:
import hashlib
ntry=1
found=False
blockNumber=1204
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:4]=='0000':
        found=True
    NONCE+=1
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

Solved  NONCE:  54671 guessHash:  00009723111ee2f2623357af34c0cee193c702eeabfe5101a86e01fae3311d6c


### 3-2 찾고자 하는 해시가 ```00000```로 시작한다고 하자. 몇 회만에 찾는지 출력하세요.

In [139]:
import hashlib
ntry=1
found=False
blockNumber=1204
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:5]=='00000':
        found=True
    NONCE+=1
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

Solved  NONCE:  963227 guessHash:  0000030a7730401222cc78b6e71c47f04b7d6b8f859479ef15c4f150167ab469


### 3-3 찾고자 하는 해시가 ```000000```로 시작한다고 하자. 몇 회만에 찾는지 출력하세요.

In [140]:
import hashlib
ntry=1
found=False
blockNumber=1204
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:6]=='000000':
        found=True
    NONCE+=1
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

Solved  NONCE:  41197733 guessHash:  0000004d5165fc6ba43ef03c699bf6fb81a76ed5daf6ea67c515eda2b75491eb


### 난이도가 어떤 경우가 높았으며, 난이도에 따라 찾는 회수의 차이가 있는지 설명하세요.
### "000000"을 찾는경우(3-3)에 난이도가 가장 높았다
### 난이도에 따른 횟수의 차이 : 난이도가 높을수록 횟수(Nonce값) 또한 증가하였다